# GOR.jl - Development

In [1]:
using GOR
using DataFrames

┌ Info: Precompiling GOR [7e057e01-19b1-480a-9799-281a76ed4e64]
└ @ Base loading.jl:1662


Idea is to develop functions that work on iterators of genome-sorted NamedTuples. The output should be Tables.jl compatible.

The simplest attempt is to work with generators

In [ ]:
select(stream, cols) = (NamedTuple{cols}(x) for x in stream)
where(stream, pred) = (x for x in stream if pred(x))
mutate(stream, func) = (merge(x, func(x)) for x in stream)
